# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

In [3]:
def read_col_file(filename: str) -> tuple:
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
        return (vertices, edges)

In [125]:
import random

def cut_cost(L: set, R: set, edges:set) -> int:
        ans = 0
        for edge in edges:
            v1, v2 = edge
            if (v1 in L and v2 in R) or (v1 in R and v2 in L):
                ans += 1
        return ans

def random_partition(vertices):
    L = set(random.sample(vertices, len(vertices) // 2))
    R = vertices - L
    
    return (L, R)

# в качестве 1-окрестности я рассматриваю разрезы
# отличающиеся от текущего в не более чем в 2 вершинах
# 2 -- если всего четное число вершин
# 1 -- иначе
def basic_local_search(graph: tuple) -> tuple:
    
    vertices, edges = graph
    n = len(vertices)
    L, R = random_partition(vertices)
    cur_cut_cost = cut_cost(L, R, edges)
    
    while True:
        min_new_L = []
        min_new_R = []
        min_new_cut_cost = cur_cut_cost

        # перебираем 2 вершины, которые мы поменяем местами
        for v1 in L:
            for v2 in R:
                new_L = L.copy()
                new_R = R.copy()
                
                new_L.remove(v1)
                new_R.remove(v2)
                new_L.add(v2)
                new_R.add(v1)
                
                cur_cut_cost = cut_cost(new_L, new_R, edges)
                if cur_cut_cost < min_new_cut_cost:
                    min_new_L = new_L
                    min_new_R = new_R
                    min_new_cut_cost = cur_cut_cost
        
        if min_new_L == []:
            return (L, R)
        else:
            L = min_new_L
            R = min_new_R
            if (len(L) > len(R)):
                L, R = R, L
            cur_cut_cost = min_new_cut_cost

In [79]:
# Запустите эту ячейку, чтобы скачать файлы с графами
# Если все сломалось, пробуйте, пока не получится

import urllib.request

url_template = 'http://mat.gsia.cmu.edu/COLOR/instances/myciel{num}.col'


for i in range(3, 8):
    with open("myciel{num}.col".format(num=i), "w") as file:
        url = url_template.format(num=i)
        response = urllib.request.urlopen(url)
        print(response.read().decode('utf-8'), file=file)


In [126]:
for i in range(3, 8):
    print("Myciel{num}:".format(num=i))
    
    vertices, edges = read_col_file('myciel{num}.col'.format(num=i))
    L, R = basic_local_search((vertices, edges))
    cut = cut_cost(L, R, edges)
    
    print(L, '\n', R, '\n', "cut cost = ", cut, '\n')

Myciel3:
{1, 6, 7, 9, 11} 
 {2, 3, 4, 5, 8, 10} 
 cut cost =  8 

Myciel4:
{1, 6, 8, 9, 13, 15, 16, 18, 20, 22, 23} 
 {2, 3, 4, 5, 7, 10, 11, 12, 14, 17, 19, 21} 
 cut cost =  28 

Myciel5:
{3, 4, 6, 7, 8, 9, 10, 13, 16, 18, 21, 22, 23, 24, 26, 34, 35, 36, 37, 38, 39, 44, 45} 
 {1, 2, 5, 11, 12, 14, 15, 17, 19, 20, 25, 27, 28, 29, 30, 31, 32, 33, 40, 41, 42, 43, 46, 47} 
 cut cost =  93 

Myciel6:
{2, 4, 5, 8, 9, 15, 19, 20, 23, 24, 28, 29, 31, 35, 37, 40, 42, 46, 47, 49, 52, 59, 60, 61, 62, 63, 64, 66, 67, 69, 71, 75, 76, 78, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 92, 95} 
 {1, 3, 6, 7, 10, 11, 12, 13, 14, 16, 17, 18, 21, 22, 25, 26, 27, 30, 32, 33, 34, 36, 38, 39, 41, 43, 44, 45, 48, 50, 51, 53, 54, 55, 56, 57, 58, 65, 68, 70, 72, 73, 74, 77, 79, 88, 93, 94} 
 cut cost =  295 

Myciel7:
{1, 3, 6, 8, 9, 12, 16, 22, 25, 27, 30, 31, 32, 36, 38, 39, 41, 43, 45, 47, 48, 50, 53, 55, 56, 59, 60, 61, 63, 64, 69, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91,